<hr style="border:2px solid #808080"> </hr>
<center><h1 style="color:#03122E;"> Álgebra Lineal Numérica IMT2111</h1></center>
<center><h1 style="color:#173F8A;"> Capítulo 1: Eliminacion Gaussiana</h3></center>
<center><h1 style="color:#0176DE;"> Prof. Manuel A. Sánchez</h3></center>
<hr style="border:2px solid #808080"> </hr>

## Tabla de contenidos

1. [Algoritmos de sustitucion regresiva y progresiva](#Algoritmos-sustitucion)
2. [Algoritmos de eliminacion Gaussiana](#Eliminacion-Gaussiana)
3. [Ejemplo: eliminacion Gaussiana](#Ejemplo:-eliminacion-Gaussiana)
4. [Ejemplo: Factorizacion LU](#Ejemplo:-Factorizacion-LU)
5. [Ejemplo: inestabilidad sin pivoteo](#Ejemplo:-inestabilidad-sin-pivoteo)
6. [Ejemplo: worst case instability](#Ejemplo:-worst-case-instability)

In [29]:
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML
display(HTML("""<style>.output {display: flex;align-items: center;text-align: center;}</style>"""))

## Algoritmos sustitucion

- **Algoritmo de sustitucion progresiva**

- **Algoritmo de sustitucion regresiva**

In [10]:
# Forward and Backward substitution
def forward_substitution(L,b):
    '''
    Forward substitution algorithm for system L x = b
    input : L lower triangular matrix n x n
            b vector n x 1
    output: x solution of L x = b
    '''
    n = L.shape[0]; x = np.zeros(n)
    x[0] = b[0]/L[0,0]
    for i in range(1,n):
        x[i] = (b[i] - L[i,0:i]@x[0:i])/L[i,i]
    return x
def backward_substitution(U,b):
    '''
    Backward substitution algorithm for system U x = b
    input : U upper tringular matrix n x n
            b vector n x 1
    output : x solution of U x = b
    '''
    n = U.shape[0]; x = np.zeros(n)
    x[n-1] = b[n-1]/U[n-1,n-1]
    for i in range(n-2,-1,-1):
        x[i] = (b[i] - U[i,(i+1):n]@(x[(i+1):n]))/U[i,i]
    return x
# generate permutation matrix
def perm(rowpiv):
    n = rowpiv.size+1
    P = np.eye(n)
    for k in range(n-1):
        P[[k,rowpiv[k]],:] = P[[rowpiv[k], k],:]
    return P
def permb(b, rowpiv):
    n = b.size
    for k in range(n-1):
        b[[k, rowpiv[k]]] = b[[rowpiv[k],k]] 
    return b

## Eliminacion Gaussiana

- **Eliminacion Gaussiana sin pivoteo**  `` GE ``
- **Factorizacion A = LU sin pivoteo**  ``LU``

- **Eliminacion Gaussiana con pivoteo parcial** ``GEPP``
- **Factorizacion LU, PA=LU, sin pivoteo**  ``LUPP``

- **Eliminacion Gaussiana con pivoteo completo** ``GECP``
- **Factorizacion LU. PAQ = LU, con pivoteo completo**  ``LUCP``

In [11]:
# 1.
def GE(Ainput, binput):
    '''
    Eliminacion Gaussiana sin pivoteo
    Input : A nonsingular and square matrix n x n 
            b vector n x 1
    Output: x solution of the system A x = b
    '''
    A = Ainput.copy() # se va a modificar
    b = binput.copy() # se va a modificar
    # 1. Factorize A = LU
    L, U = LU(A)
    # 2. Solve LUx = b forward substitution
    y = forward_substitution(L, b)
    # 3. Solve Ux = L^{-1} b backward substitution
    x = backward_substitution(U,y)
    return x
def LU(Ainput):
    '''
    Factorizacion A = LU, sin pivoteo
    Input : A nonsingular and square matrix n x n 
    Output: L triangular inferior, U triangular superior, square matrix n x n
    '''
    A = Ainput.copy() # se va a modificar
    n = A.shape[0]
    for i in range(n-1):
        if A[i, i] == 0:
            raise ValueError("coeficient is zero.")
        A[(i+1):n,i] = (A[(i+1):n,i]/A[i,i])
        A[(i+1):n, (i+1):n][:] = A[(i+1):n, (i+1):n]-np.outer(A[(i+1):n,i],A[i, (i+1):n])
    L = np.tril(A,-1)+np.eye(n)
    U = np.triu(A)
    return L, U

# 2.
def GEPP(Ainput, binput):
    A = Ainput.copy() # se va a modificar
    b = binput.copy() # se va a modificar
    '''
    Eliminacion Gaussiana con pivoteo de filas o pivoteo parcial
    Input : A matriz cuadrada no singular de n x n 
            b vector de n x 1
    Output: x solucion del sistema lineal  A x = b
    '''
    # 1. Factorize A = PLU
    L, U, pT = LUPP(A)
    # 2. Solve P L U x = b
    P = perm(pT)
    rhs  = perm(pT).dot(b)
    # 3. Solve LUx = Pt b forward substitution
    y = forward_substitution(L, rhs)
    # 4. Solve Ux = L^{-1} Pt b backward substitution
    x = backward_substitution(U,y)
    return x

def LUPP(Ainput):
    '''
    Factorizacion PA = LU, con pivoteo parcial
    Input : A matriz cuadrada no singular de n x n
    Output: L triangular inferior, matriz cuadrada de n x n
            U trangular superior, matriz cuadrada de n x n
            p, vector asociado a la matriz de permutacion
    '''
    A = Ainput.copy() # se va a modificar
    n = A.shape[0]
    piv = np.arange(0,n-1)
    for i in range(n-1):
        imax = abs(A[i:,i]).argmax() + i
        piv[i] = imax
        if A[imax, i] == 0:
            raise ValueError("Matrix is singular.")
        elif imax != i:
            A[[i,imax],:] = A[[imax, i],:][:]
        A[(i+1):n,i][:] = (A[(i+1):n,i]/A[i,i])[:]
        A[(i+1):n, (i+1):n][:] = A[(i+1):n, (i+1):n]-np.outer(A[(i+1):n,i],A[i, (i+1):n])
    
    L = np.tril(A,-1)+np.eye(n)
    U = np.triu(A)
    return L, U, piv

# 
def GECP(Ainput, binput):
    '''
    Eliminacion Gaussiana con pivoteo de filas y columnas o pivoteo completo
    Input : A matriz cuadrada no singular de n x n 
            b vector de n x 1
    Output: x solucion del sistema lineal  A x = b
    '''
    A = Ainput.copy() # se va a modificar
    b = binput.copy() # se va a modificar
    # 1. Factorize PAQ^T = LU
    L, U, rowpiv, colpiv = LUCP(A)
    # 2. Solve  L U x = P b Q^T
    rhs  = perm(rowpiv).dot(b)
    # 3. Solve LUx = Pt b forward substitution
    y = forward_substitution(L, rhs)
    # 4. Solve Ux = L^{-1} Pt b backward substitution
    x = backward_substitution(U,y)
    return (perm(colpiv).T).dot(x)

def LUCP(Ainput):
    '''
    Factorizacion P A Q = L U, con pivoteo completo
    Input : A matriz cuadrada no singular de n x n
    Output: L triangular inferior, matriz cuadrada de n x n
            U trangular superior, matriz cuadrada de n x n
            p, vector asociado a la matriz de permutacion por filas
            q, vector asociado a la matriz de permutacion por columnas
    '''
    A = Ainput.copy() # se va a modificar
    n = A.shape[0]
    rowpiv = np.arange(0,n-1)
    colpiv = np.arange(0,n-1)
    for i in range(n-1):
        mu, lam = np.unravel_index(np.argmax(np.abs(A[i:,i:]), axis=None), A[i:,i:].shape)
        mu +=i; lam+=i
        if A[mu, lam] == 0:
            raise ValueError("Matrix is singular.")
        else:
            rowpiv[i] = mu
            A[[i, mu],:] = A[[mu, i],:][:]
            colpiv[i] = lam
            A[:,[i, lam]] = A[:,[lam, i]][:]
        A[(i+1):n,i] *= 1.0/A[i,i]
        A[(i+1):n, (i+1):n] -=np.outer(A[(i+1):n,i],A[i, (i+1):n])
    L = np.tril(A,-1)+np.eye(n)
    U = np.triu(A)
    return L, U, rowpiv, colpiv

In [12]:
def Eliminacion_Gaussiana(A,b, pivoteo=None):
    if pivoteo is None:
        x = GE(A,b)
    elif pivoteo == 'parcial':
        x = GEPP(A,b)
    elif pivoteo == 'completo':
        x = GECP(A,b)
    return x

def Factorizacion_LU(A, pivoteo=None):
    if pivoteo is None:
        L,U = LU(A)
        return L,U
    elif pivoteo == 'parcial':
        L,U,piv = LUPP(A)
        return L, U, piv
    elif pivoteo == 'completo':
        L, U, rowpiv, colpiv = LUCP(A)
        return  L, U, rowpiv, colpiv

## Ejemplo: eliminacion Gaussiana

Considere el sistema lineal
\begin{equation}
Ax = b,\qquad 
A = \begin{bmatrix}
1 & 3 & 4 & 1\\
2 & 1 & 5 & 1 \\
3 & 1 & 6 & 1 \\
6 & 2 & 3 & 2
\end{bmatrix},\quad
b = \begin{bmatrix}
-2 \\ -2\\ -2 \\ 5
\end{bmatrix}
\end{equation}

Resuelva usando Eliminacion Gaussiana, i) sin pivoteo, ii) con pivoteo parcial, iii) con pivoteo completo.

In [15]:
A = np.array([[1,3,4,1],[2,1,5,1],[3,1,6,1],[6,2,3,2]],dtype=np.float64)
x = np.array([1, 0, -1,1])
b = np.array([-2,-2,-2,5],dtype=np.float64)

In [16]:
x_GE   = Eliminacion_Gaussiana(A,b)
x_GEPP = Eliminacion_Gaussiana(A,b, pivoteo='parcial')
x_GECP = Eliminacion_Gaussiana(A,b, pivoteo='completo')
print("x_GE   : ", x_GE)
print("x_GEPP : ", x_GEPP)
print("x_GECP : ", x_GECP)
print("x      : ", np.linalg.solve(A,b))


x_GE   :  [ 1.00000000e+00  2.66453526e-16 -1.00000000e+00  1.00000000e+00]
x_GEPP :  [ 1.  0. -1.  1.]
x_GECP :  [ 1.00000000e+00  3.74700271e-16 -1.00000000e+00  1.00000000e+00]
x      :  [ 1.  0. -1.  1.]


## Ejemplo: Factorizacion LU

Encuentre la factorizacion LU de la siguiente matriz usando los algoritmos de Eliminacion Gaussiana, i) sin pivoteo, ii) con pivoteo parcial, iii) con pivoteo completo. Compare las matrices:
\begin{equation}
LU = A,\qquad 
A = \begin{bmatrix}
1 & 3 & 4 & 1\\
2 & 1 & 5 & 1 \\
3 & 1 & 6 & 1 \\
6 & 2 & 3 & 2
\end{bmatrix},
\end{equation}

In [17]:
A = np.array([[1,3,4,1],[2,1,5,1],[3,1,6,1],[6,2,3,2]],dtype=np.float64)

In [18]:
L, U = Factorizacion_LU(A, pivoteo=None)
print("L :\n", L)
print("U :\n", U)

L :
 [[1.  0.  0.  0. ]
 [2.  1.  0.  0. ]
 [3.  1.6 1.  0. ]
 [6.  3.2 9.5 1. ]]
U :
 [[ 1.   3.   4.   1. ]
 [ 0.  -5.  -3.  -1. ]
 [ 0.   0.  -1.2 -0.4]
 [ 0.   0.   0.   3. ]]


In [19]:
L, U, p = Factorizacion_LU(A, pivoteo='parcial')
print("L :\n", L)
print("U :\n", U)

L :
 [[1.         0.         0.         0.        ]
 [0.16666667 1.         0.         0.        ]
 [0.5        0.         1.         0.        ]
 [0.33333333 0.125      0.79166667 1.        ]]
U :
 [[6.         2.         3.         2.        ]
 [0.         2.66666667 3.5        0.66666667]
 [0.         0.         4.5        0.        ]
 [0.         0.         0.         0.25      ]]


In [20]:
L, U,p,q = Factorizacion_LU(A, pivoteo='completo')
print("L :\n", L)
print("U :\n", U)

L :
 [[ 1.          0.          0.          0.        ]
 [ 0.5         1.          0.          0.        ]
 [ 0.66666667 -0.22222222  1.          0.        ]
 [ 0.83333333 -0.11111111  0.125       1.        ]]
U :
 [[6.         3.         1.         1.        ]
 [0.         4.5        1.5        1.5       ]
 [0.         0.         2.66666667 0.66666667]
 [0.         0.         0.         0.25      ]]


## Ejemplo: inestabilidad sin pivoteo
\begin{equation}
A = \begin{bmatrix}
0 & 1 \\
1 & 1
\end{bmatrix}, \quad \text{Eliminacion Gaussiana falla en el primer paso con } \kappa(A) = (3+\sqrt{5})/2
\end{equation}

\begin{equation}
\tilde{A} = \begin{bmatrix}
10^{-20} & 1 \\
1 & 1
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 \\
10^{20} & 1
\end{bmatrix}
\begin{bmatrix}
10^{-20} & 1 \\
0 & 1-10^{20}
\end{bmatrix}, \quad \text{Eliminacion Gaussiana ahora no falla}
\end{equation}

In [21]:
Afalla = np.array([[0,1],[1,1]], dtype=np.float64)
Atilde = np.array([[10**(-20), 1],[1,1]], dtype=np.float64)

In [22]:
L,U = Factorizacion_LU(Afalla)


ValueError: coeficient is zero.

In [23]:
L,U = Factorizacion_LU(Atilde)
print(L)
print(U)
print(L@U- Atilde)

[[1.e+00 0.e+00]
 [1.e+20 1.e+00]]
[[ 1.e-20  1.e+00]
 [ 0.e+00 -1.e+20]]
[[ 0.  0.]
 [ 0. -1.]]


In [24]:
b = np.array([1,0])
xtilde = Eliminacion_Gaussiana(Atilde,b)
print("solucion calculada:", xtilde)
print("la solucion exacta es x = [-1,1]!")

solucion calculada: [0. 1.]
la solucion exacta es x = [-1,1]!


**Nota** La factorizacion LU es estable, no backward stable. Pero al usar eliminacion Gaussiana sin pivoteo para resolver $Ax=b$ no lo hace estable.

En general, si un paso del algoritmo es estable pero backward stable para resolver un subproblema, entonces la estabilidad del algortimo puede estar en peligro.

**Nota** Complejidad:  eliminacion Gaussiana sin pivoteo es de $\approx \frac{2}{3} m^{3}$ flops.

## Ejemplo: worst case instability

### Inestabilidad

Para ciertas matrices $A$ a pesar de os efectos beneficiosos de pivotear, el factor de crecimiento $\rho$ se vuelve gigante. Por ejemplo, suponga que la matriz $A$ tiene la siguiente forma:

$$
A = \begin{bmatrix}
1 & 0 & 0 & 0 & 1 \\
-1 &1 &0 & 0& 1 \\
-1& -1 &1 & 0 &1\\
-1&-1 &-1 & 1 & 1 \\
-1& -1 &-1 &-1 &1
\end{bmatrix}
$$

En este caso, la factorization da 

$$
U = \begin{bmatrix}
 1. & 0. & 0. & 0.&  1.\\
 0. & 1. & 0. & 0.&  2.\\
 0. & 0. & 1. & 0.&  4.\\
 0. & 0. & 0. & 1.&  8.\\
 0. & 0. & 0. & 0.& 16.
 \end{bmatrix}
$$

para esta matriz de $n\times n$, con $n=5$, el factor de crecimiento es $\rho = 2^{n-1} = 16$.

Un factor de crecimiento de orden $2^{n}$ corresponde a una perdida del orden de $n$ bits de precision, lo cual es catastrofico para computaciones practicas. Como un computador standard representa numeros de punto flotante con solo 64 bits, con matries de dimensiones en cientos y miles en dimension perder m bits de precision es intolerable.

In [31]:
n = 4
A_wc = -np.tril(np.ones((n,n)),-1)+np.eye(n)
A_wc[:,-1] = np.ones(n)
A_wc

array([[ 1.,  0.,  0.,  1.],
       [-1.,  1.,  0.,  1.],
       [-1., -1.,  1.,  1.],
       [-1., -1., -1.,  1.]])

In [32]:
L, U, pT = Factorizacion_LU(A_wc, pivoteo='parcial')
U

array([[1., 0., 0., 1.],
       [0., 1., 0., 2.],
       [0., 0., 1., 4.],
       [0., 0., 0., 8.]])

In [33]:
L, U, pT, q = Factorizacion_LU(A_wc, pivoteo='completo')
U

array([[ 1.,  1.,  0.,  0.],
       [ 0.,  2.,  1.,  0.],
       [ 0.,  0., -2.,  1.],
       [ 0.,  0.,  0., -2.]])

In [36]:
e1 = []; e2 = []; e3 = []; e4 = []; N = [5*i for i in range(1,8)]
for nn in N:
    A_wc = -np.tril(np.ones((nn,nn)),-1)+np.eye(nn); A_wc[:,-1] = np.ones(nn)    
    x = np.random.rand(nn); b = A_wc.dot(x)

    x_gepp = Eliminacion_Gaussiana(A_wc, b, pivoteo='parcial')
    x_gecp = Eliminacion_Gaussiana(A_wc, b, pivoteo='completo')

    e1.append(np.linalg.norm(x-x_gepp))
    e2.append(np.linalg.norm(x-x_gecp))
    e3.append(np.linalg.norm(b-A_wc.dot(x_gepp)))
    e4.append(np.linalg.norm(b-A_wc.dot(x_gecp)))

dferror = {'n':N, '||x-x_gepp||':e1, '||x-x_gecp||':e2, '||b-A x_gepp||':e3, '||b - A x_gecp||':e4}
df = pd.DataFrame(data=dferror)
df

,n,||x-x_gepp||,||x-x_gecp||,||b-A x_gepp||,||b - A x_gecp||
0,5,3.723801e-16,2.220446e-16,6.329245e-16,0.000000e+00
1,10,2.621725e-14,3.608225e-16,4.794223e-14,6.753223e-16
2,15,2.328402e-13,9.730299e-16,3.498509e-13,2.175584e-15
3,20,3.148603e-11,8.400361e-16,5.750045e-11,5.087681e-16
4,25,5.447583e-10,1.735334e-15,8.676726e-10,8.968101e-16
5,30,4.397691e-09,2.149938e-15,6.778379e-09,3.244911e-15
6,35,2.595358e-07,2.950066e-15,4.374213e-07,6.179466e-15
